In [1]:
# For now, clear outputs (Cell > All Output > Clear) before committing to Git
# There might be a better way

import sqlite3
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing
import sklearn.ensemble as ske
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

/Users/Niclas/vortrag/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Read wildfire data from SQLITE database
cnx = sqlite3.connect('FPA_FOD_20170508.sqlite')
df = pd.read_sql_query("SELECT FIRE_YEAR,DISCOVERY_TIME,STAT_CAUSE_DESCR,CONT_DATE,CONT_TIME,LATITUDE,LONGITUDE,STATE,DISCOVERY_DATE,FIRE_SIZE,FIRE_SIZE_CLASS FROM 'Fires'", cnx)
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous  2453403.5      1730  40.036944   
1       2004           0845        Lightning  2453137.5      1530  38.933056   
2       2004           1921   Debris Burning  2453156.5      2024  38.984167   
3       2004           1600        Lightning  2453189.5      1400  38.559167   
4       2004           1600        Lightning  2453189.5      1200  38.559167   

    LONGITUDE STATE  DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA       2453403.5       0.10               A  
1 -120.404444    CA       2453137.5       0.25               A  
2 -120.735556    CA       2453156.5       0.10               A  
3 -119.913333    CA       2453184.5       0.10               A  
4 -119.933056    CA       2453184.5       0.10               A  


In [3]:
# Convert fire discovery and containment dates to yyyy-mm-dd
df['DISCOVERY_DATE'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
df['CONT_DATE'] = pd.to_datetime(df['CONT_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  
0 -121.005833    CA     2005-02-02       0.10               A  
1 -120.404444    CA     2004-05-12       0.25               A  
2 -120.735556    CA     2004-05-31       0.10               A  
3 -119.913333    CA     2004-06-28       0.10               A  
4 -119.933056    CA     2004-06-28       0.10               A  


In [4]:
# Add new columns for month, date, and weekday for discovery and containment dates

df['DISCOVERY_MONTH'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).month
df['DISCOVERY_DAY'] = pd.DatetimeIndex(df['DISCOVERY_DATE']).day
df['DISCOVERY_DAY_OF_WEEK'] = df['DISCOVERY_DATE'].dt.weekday_name

#df['CONT_DATE'].fillna(0)
df['CONT_MONTH'] = pd.DatetimeIndex(df['CONT_DATE']).month
df['CONT_DAY'] = pd.DatetimeIndex(df['CONT_DATE']).day
df['CONT_DAY_OF_WEEK'] = df['CONT_DATE'].dt.weekday_name
print(df.head())

   FIRE_YEAR DISCOVERY_TIME STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300    Miscellaneous 2005-02-02      1730  40.036944   
1       2004           0845        Lightning 2004-05-12      1530  38.933056   
2       2004           1921   Debris Burning 2004-05-31      2024  38.984167   
3       2004           1600        Lightning 2004-07-03      1400  38.559167   
4       2004           1600        Lightning 2004-07-03      1200  38.559167   

    LONGITUDE STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833    CA     2005-02-02       0.10               A   
1 -120.404444    CA     2004-05-12       0.25               A   
2 -120.735556    CA     2004-05-31       0.10               A   
3 -119.913333    CA     2004-06-28       0.10               A   
4 -119.933056    CA     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY DISCOVERY_DAY_OF_WEEK  CONT_MONTH  CONT_DAY  \
0                2              2             W

In [5]:
le = preprocessing.LabelEncoder()
df['STAT_CAUSE_DESCR'] = le.fit_transform(df['STAT_CAUSE_DESCR'])
df['STATE'] = le.fit_transform(df['STATE'])
df['DISCOVERY_DAY_OF_WEEK'] = le.fit_transform(df['DISCOVERY_DAY_OF_WEEK'])
print(df.head())

   FIRE_YEAR DISCOVERY_TIME  STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300                 7 2005-02-02      1730  40.036944   
1       2004           0845                 6 2004-05-12      1530  38.933056   
2       2004           1921                 3 2004-05-31      2024  38.984167   
3       2004           1600                 6 2004-07-03      1400  38.559167   
4       2004           1600                 6 2004-07-03      1200  38.559167   

    LONGITUDE  STATE DISCOVERY_DATE  FIRE_SIZE FIRE_SIZE_CLASS  \
0 -121.005833      4     2005-02-02       0.10               A   
1 -120.404444      4     2004-05-12       0.25               A   
2 -120.735556      4     2004-05-31       0.10               A   
3 -119.913333      4     2004-06-28       0.10               A   
4 -119.933056      4     2004-06-28       0.10               A   

   DISCOVERY_MONTH  DISCOVERY_DAY  DISCOVERY_DAY_OF_WEEK  CONT_MONTH  \
0                2              2           

In [6]:
df['CONT_DAY_OF_WEEK']=df['CONT_DAY_OF_WEEK'].fillna("Unknown")
df['CONT_DAY']=df['CONT_DAY'].fillna("0")
df['CONT_MONTH']=df['CONT_MONTH'].fillna("0")
df['CONT_TIME']=df['CONT_TIME'].fillna("0")
df['DISCOVERY_TIME']=df['DISCOVERY_TIME'].fillna("0")

df['CONT_DAY_OF_WEEK'] = le.fit_transform(df['CONT_DAY_OF_WEEK'])
df['FIRE_SIZE_CLASS'] = le.fit_transform(df['FIRE_SIZE_CLASS'])
print(df.head())

   FIRE_YEAR DISCOVERY_TIME  STAT_CAUSE_DESCR  CONT_DATE CONT_TIME   LATITUDE  \
0       2005           1300                 7 2005-02-02      1730  40.036944   
1       2004           0845                 6 2004-05-12      1530  38.933056   
2       2004           1921                 3 2004-05-31      2024  38.984167   
3       2004           1600                 6 2004-07-03      1400  38.559167   
4       2004           1600                 6 2004-07-03      1200  38.559167   

    LONGITUDE  STATE DISCOVERY_DATE  FIRE_SIZE  FIRE_SIZE_CLASS  \
0 -121.005833      4     2005-02-02       0.10                0   
1 -120.404444      4     2004-05-12       0.25                0   
2 -120.735556      4     2004-05-31       0.10                0   
3 -119.913333      4     2004-06-28       0.10                0   
4 -119.933056      4     2004-06-28       0.10                0   

   DISCOVERY_MONTH  DISCOVERY_DAY  DISCOVERY_DAY_OF_WEEK CONT_MONTH CONT_DAY  \
0                2            

In [7]:
df['CONT_MONTH']=df['CONT_MONTH'].astype('Float64')
df['CONT_DAY']=df['CONT_DAY'].astype('Float64')
#df['CONT_TIME']=df['CONT_TIME'].astype('Float64')
#df['DISCOVERY_DATE']=df['DISCOVERY_DATE'].astype('Float64')

In [8]:
df=df.drop(['CONT_DATE','CONT_TIME','DISCOVERY_TIME','STATE','FIRE_SIZE'],axis=1)

ml for severity

In [ ]:
labels=df['FIRE_SIZE_CLASS']
labels.head()

In [ ]:
logits=df.drop(['FIRE_SIZE','FIRE_SIZE_CLASS','DISCOVERY_DATE','CONT_DATE','STATE','CONT_TIME','DISCOVERY_TIME'],axis=1)
logits.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels,test_size=0.2)

In [ ]:
reg = LinearRegression().fit(x_train, y_train)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
x_test.first

In [ ]:
#reg.predict(x_test[1])

In [ ]:
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization
from keras.optimizers import SGD

In [ ]:
from keras.activations import relu

In [ ]:
labels_cat=keras.utils.to_categorical(labels,7)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(logits,labels_cat,test_size=0.2)

In [ ]:
labels_cat.shape

In [ ]:
model=Sequential()
model.add(BatchNormalization(input_shape=[10]))
model.add(Dense(500,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(300,kernel_initializer='truncated_normal'))
model.add(Activation('sigmoid'))
model.add(Dense(7,kernel_initializer='truncated_normal'))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer=SGD(lr=0.01, momentum=0.9,decay=1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
model.fit(x_train,y_train,epochs=10,validation_data=(np.array(x_test),np.array(y_test)))

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
a=model.predict(x_test)

Generating Air quality data

In [9]:
def preprocess_site_list(site_file):
    # Make sure that state, county, and site codes are read as strings
    # instead of numbers
    col_types = {'State Code': str, 'County Code': str, 'Site Number': str}

    sites = pd.read_csv(site_file, dtype=col_types)

    # Columns that can be dropped from the site listing
    cols_to_drop = [
        'Land Use', 'Location Setting', 'Met Site State Code',
        'Met Site County Code', 'Met Site Site Number', 'Met Site Type',
        'Met Site Distance', 'Met Site Direction', 'GMT Offset',
        'Owning Agency', 'Local Site Name', "Address", "Zip Code",
        "State Name", "County Name", "City Name", "CBSA Name", "Tribe Name",
        "Extraction Date"
    ]

    sites = sites.drop(columns=cols_to_drop)

    # Convert site closing dates to datetime64 objects (does not work
    # with read_csv())
    sites['Site Closed Date'] = pd.to_datetime(sites['Site Closed Date'])

    # Get sites that have been closed down before 1992 and drop them
    # from the list
    outdated = sites.loc[
        sites['Site Closed Date'] < pd.to_datetime('01-01-1992')]
    sites = sites.drop(outdated.index)

    # Get the complete site code, add it as a column, and finally
    # use the site code as the index
    sites['site-code'] = sites['State Code'] + '-' + sites[
        'County Code'] + '-' + sites['Site Number']
    sites = sites.set_index('site-code')

    # Divide the data into two Series: one with latitude coordinates and one
    # with longitude coordinates
    sites_lat = sites['Latitude']
    sites_lon = sites['Longitude']

    # Transform the site listings into a dictionary where keys are site
    # codes and values are coordinates}
    sites_lat = sites_lat.to_dict()
    sites_lon = sites_lon.to_dict()

    # Return the two dictionaries with the coordinates
    return sites_lat, sites_lon


def preprocess_aqi(sites_lat, sites_lon, aqi_file):
    # Read daily AQI measurements from year XXXX
    
    # Specify used column names and types to make CSV parsing more efficient
    col_types = {'AQI': np.int32, 'Category': str, 'Defining Parameter': str,
                 'Defining Site': str}
    
    col_names = list(col_types.keys()) + ['Date']
    
    aqi = pd.read_csv(aqi_file, usecols=col_names, dtype=col_types, parse_dates=['Date'])

    # Add column containing the site information (including location
    # coordinates) to each AQI measurement
    aqi['Latitude'] = aqi['Defining Site'].map(sites_lat)
    aqi['Longitude'] = aqi['Defining Site'].map(sites_lon)

    return aqi


In [11]:
# Get dictionaries containing latitude and longitude coordinates for AQI measurement sites
sites_lat, sites_lon = preprocess_site_list('data/aqi/aqs_sites.csv')

# Generate a list of years 
years = [str(y) for y in range(1992, 2016)]

# Generate a dictionary where keys are years (as strings) and values are DataFrames
# containing the AQI data with coordinates added
aqi = {y: preprocess_aqi(sites_lat, sites_lon, 'data/aqi/daily_aqi_by_county_{}.csv'.format(y)) for y in years}

# Combine all AQI data into a single large DataFrame (~7 million rows)
aqi_all = pd.concat([aqi[y] for y in aqi.keys()], axis=0, ignore_index=True)

#aqi_all

In [12]:
fire_lat = df['LATITUDE']
fire_lon = df['LONGITUDE']
dis_date = df['DISCOVERY_DATE']
new_df = pd.DataFrame()

for i in range(df.shape[0]):
    #print(i)
    
    a=df.iloc[i]['FIRE_YEAR']    
    aqiframe=aqi[str(a)]

    
    measure_date = aqiframe['Date']
    aqi_lat = aqiframe['Latitude']
    aqi_lon = aqiframe['Longitude']
    
    MinDistance = 10000000000000
    MinDistanceColumn = None

    fitting_locations = []
    for j in range(aqiframe.shape[0]): 
        Distance = ((fire_lat.iloc[i]-aqi_lat.iloc[j])**2 + (fire_lon.iloc[i]-aqi_lon.iloc[j])**2)**0.5  
        Distance = round(Distance, 6)

        if Distance < MinDistance:
            MinDistance = Distance
            fitting_locations = []
            fitting_locations.append(j)
            
        elif Distance == MinDistance:
            fitting_locations.append(j)

    for k in fitting_locations:
        if dis_date.iloc[i] < measure_date.astype('datetime64').iloc[k]:
            right_column = aqiframe.iloc[k]
            break
            
    new_series = pd.concat([df.iloc[i],right_column])
    new_df=pd.concat([new_df,new_series],axis=1)

0
2005


/Users/Niclas/vortrag/lib/python3.6/site-packages/ipykernel_launcher.py:59: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



1
2004
2
2004
3
2004
4
2004


KeyboardInterrupt: 

In [13]:
new_df2=new_df.swapaxes(0,1)
print(new_df2)


   AQI CONT_DAY CONT_DAY_OF_WEEK CONT_MONTH                        Category  \
0   68        2                7          2                        Moderate   
0   68       12                7          5                        Moderate   
0  112       31                1          5  Unhealthy for Sensitive Groups   
0   64        3                2          7                        Moderate   

        DISCOVERY_DATE DISCOVERY_DAY DISCOVERY_DAY_OF_WEEK DISCOVERY_MONTH  \
0  2005-02-02 00:00:00             2                     6               2   
0  2004-05-12 00:00:00            12                     6               5   
0  2004-05-31 00:00:00            31                     1               5   
0  2004-06-28 00:00:00            28                     1               6   

                  Date Defining Parameter Defining Site FIRE_SIZE_CLASS  \
0  2005-02-03 00:00:00              PM2.5   06-063-1006               0   
0  2005-02-03 00:00:00              PM2.5   06-063-1006        

In [14]:
new_df2.columns

Index(['AQI', 'CONT_DAY', 'CONT_DAY_OF_WEEK', 'CONT_MONTH', 'Category',
       'DISCOVERY_DATE', 'DISCOVERY_DAY', 'DISCOVERY_DAY_OF_WEEK',
       'DISCOVERY_MONTH', 'Date', 'Defining Parameter', 'Defining Site',
       'FIRE_SIZE_CLASS', 'FIRE_YEAR', 'LATITUDE', 'LONGITUDE', 'Latitude',
       'Longitude', 'STAT_CAUSE_DESCR'],
      dtype='object')

In [18]:
drop=new_df2.drop(['DISCOVERY_DATE','Date','LATITUDE','LONGITUDE','Latitude','Longitude','STAT_CAUSE_DESCR'],axis=1)

In [19]:
drop=drop.drop(['Defining Parameter','Defining Site','Category'],axis=1)

In [20]:
drop.head()

,AQI,CONT_DAY,CONT_DAY_OF_WEEK,CONT_MONTH,DISCOVERY_DAY,DISCOVERY_DAY_OF_WEEK,DISCOVERY_MONTH,FIRE_SIZE_CLASS,FIRE_YEAR
0,68,2,7,2,2,6,2,0,2005
0,68,12,7,5,12,6,5,0,2004
0,112,31,1,5,31,1,5,0,2004
0,64,3,2,7,28,1,6,0,2004


In [22]:
labels=drop['AQI']

In [23]:
logits=drop.drop(['AQI'],axis=1)

In [24]:
labels.head()

0     68
0     68
0    112
0     64
Name: AQI, dtype: object

In [25]:
logits.head()

,CONT_DAY,CONT_DAY_OF_WEEK,CONT_MONTH,DISCOVERY_DAY,DISCOVERY_DAY_OF_WEEK,DISCOVERY_MONTH,FIRE_SIZE_CLASS,FIRE_YEAR
0,2,7,2,2,6,2,0,2005
0,12,7,5,12,6,5,0,2004
0,31,1,5,31,1,5,0,2004
0,3,2,7,28,1,6,0,2004


In [ ]:
import xgboost as xgb

In [ ]:
clf = xgb.XGBModel()

clf.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        eval_metric='logloss',
        verbose=True)

ypred = clf.predict(x_test)
print(ypred)

In [ ]:
df['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_arson = df[df['STAT_CAUSE_DESCR']=='Arson']
df_arson['DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
def plot_corr(df,size=10):
    corr = df.corr()  #the default method is pearson
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr,cmap=plt.cm.Oranges)
    plt.xticks(range(len(corr.columns)), corr.columns)
    plt.yticks(range(len(corr.columns)), corr.columns)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)    
    plt.show()        
plot_corr(df)

In [ ]:
dfplot=df

In [ ]:
dfplot=dfplot.sort_index(by=['FIRE_SIZE'],axis=0, ascending=False)

In [ ]:
dflol=dfplot[:500]

In [ ]:
dflol.plot(kind='scatter',x='LONGITUDE',y='LATITUDE',color='coral',alpha=0.3)
plt.show()

In [ ]:
df.head()

In [ ]:
df_lightning = df#[df['STAT_CAUSE_DESCR']=='Lightning']
df_lightning['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='barh',color='coral')
plt.show()

In [ ]:
df['STATE'].value_counts().head(n=10).plot(kind='barh',color='coral')
plt.show()

In [ ]:
df_CA = df[df['STATE']=='CA']
df_CA['STAT_CAUSE_DESCR'].value_counts().plot(kind='barh',color='coral',title='causes of fires for CA')
plt.show()